In [1]:
gpu_info_examine = !nvidia-smi
gpu_info_examine = '\n'.join(gpu_info_examine)
if gpu_info_examine.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info_examine)

Wed Jul 14 07:50:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install tqdm
!pip install numpy

     |████████████████████████████████| 2.5MB 13.9MB/s 
     |████████████████████████████████| 3.3MB 59.5MB/s 
     |████████████████████████████████| 901kB 49.1MB/s 
     |████████████████████████████████| 266kB 13.9MB/s 
     |████████████████████████████████| 122kB 65.6MB/s 
     |████████████████████████████████| 245kB 57.4MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, BartTokenizerFast
from datasets import load_dataset, load_metric
import torch as torch
from tqdm import tqdm
from datasets import load_metric
import numpy as np

In [5]:
model_checkpoint = 'a1noack/bart-large-gigaword' #'facebook/bart-base'  #'facebook/bart-large-cnn'
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


cuda


In [6]:
# test = load_dataset("wikihow", "all", data_dir="/scratch/nm3571", split='test')
# test = load_dataset("gigaword", split='test')
# test = load_dataset("big_patent", "g", split='test')
# test = load_dataset("xsum", split='test')
test = load_dataset("big_patent", "g", split='test') #,cache_dir='/scratch/yk2516/cache/')
# test = load_dataset("cnn_daily",cache_dir='/scratch/yk2516/cache/', split='test[:]')
# test = load_dataset("cnn_dailymail", name='3.0.0',split='test', cache_dir='/scratch/yk2516/cache/')

Dataset big_patent downloaded and prepared to /root/.cache/huggingface/datasets/big_patent/g/1.0.0/efa16ff728ce0a1726ef8a0faeb0376331093f8fff41cf4cfaccc11d9cdb442d. Subsequent calls will reuse this data.


In [7]:
print(test)
print(model.config.max_length)

Dataset({
    features: ['description', 'abstract'],
    num_rows: 14386
})
32


In [8]:
encodings =  tokenizer(test['description'], return_tensors='pt', padding=True, truncation=True, max_length=1024).to(device)

In [9]:
model = model.to(device)
model.eval()
number_beams = 8
# print(encodings['input_ids'])
# print(encodings['input_ids'].cpu().detach().numpy())
# print(torch.from_numpy(encodings['input_ids'].cpu().detach().numpy()).to(device))
# print(type(encodings['input_ids'][0].item()))
# print("Wikihow vocab size: ", result.scores[0].shape[1])
print("Input ids size", encodings['input_ids'].shape)
ids = encodings['input_ids'].cpu().detach().numpy()
attention_ids = encodings['attention_mask'].cpu().detach().numpy()
log_sent = []
num_words = []
print("number of samples:", ids.shape[0])

Input ids size torch.Size([14386, 1024])
number of samples: 14386


In [10]:
!pip install rouge_score

In [11]:
rouge = load_metric('rouge')


In [12]:
ids.shape[0]

14386

In [16]:
for i in range(ids.shape[0]):
    print(i)
    input_id = np.array([ids[i]])
    input_id = torch.from_numpy(input_id).to(device)
    attention_id = np.array([attention_ids[i]])
    attention_id = torch.from_numpy(attention_id).to(device)
    with torch.no_grad():
        result = model.generate(input_ids=input_id, attention_mask=attention_id, num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)
    
    generated_summary = tokenizer.batch_decode(result[0], skip_special_tokens=True) #, clean_up_tokenization_spaces=False)
    # rouge.add_batch(predictions=generated_summary, references=test['summary'][i])
    rouge.add(prediction=generated_summary[0], reference=test['abstract'][i])
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


KeyboardInterrupt: ignored

In [ ]:
score = rouge.compute()

In [ ]:
print('Rouge: ')
print(score)